In [1]:
import random
import pandas as pd
import re
import jieba
import numpy
from collections import Counter
from operator import itemgetter

In [2]:
salesman="""
salesman = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 买 具体业务 保险
玩玩 = 耍一耍 | 玩一玩
具体业务 = 人寿 | 残疾 | 旅行 | 车 | 财产
结尾 = 吗？
"""


In [3]:
def create_grammar(grammar_str,split='=',line_split='\n'):
    grammar={}  #字典
    for line in grammar_str.split(line_split):
        if not line.strip():continue
        exp, stmt = line.split(split)
        grammar[exp.strip()]=[s.split() for s in stmt.split('|')]
    return grammar


In [4]:
choice=random.choice
def generate(gram,target):
    if target not in gram: return target #terminal expression
    expaned=[generate(gram,t)for t in choice(gram[target])]
    return ''.join([e  if e!='/n' else '\n' for e in expaned if e!='null'])
example_grammar=create_grammar(salesman)

In [5]:
example_grammar

{'salesman': [['寒暄', '报数', '询问', '业务相关', '结尾']],
 '报数': [['我是', '数字', '号', ',']],
 '数字': [['单个数字'], ['数字', '单个数字']],
 '单个数字': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']],
 '寒暄': [['称谓', '打招呼'], ['打招呼']],
 '称谓': [['人称', ',']],
 '人称': [['先生'], ['女士'], ['小朋友']],
 '打招呼': [['你好'], ['您好']],
 '询问': [['请问你要'], ['您需要']],
 '业务相关': [['买', '具体业务', '保险']],
 '玩玩': [['耍一耍'], ['玩一玩']],
 '具体业务': [['人寿'], ['残疾'], ['旅行'], ['车'], ['财产']],
 '结尾': [['吗？']]}

In [6]:
filename='train.txt'
def getfile(filename):
    content=""
    with open(filename, 'r',encoding='UTF-8') as file_to_read:
        while True:
            lines = file_to_read.readline()
            if not lines: break
            s1, s2,s3,s4 = [i for i in lines.split('++$++')]
            content = content+s3.strip()
    return content
def token(string):
    return re.findall('\w+',string)  #正则表达式
def cut(string):
    return list(jieba.cut(string))
quesitons=''.join(token(getfile(filename)))

In [8]:
quesitons

In [9]:
with_jieba_cut=Counter(jieba.cut(quesitons))
print(with_jieba_cut.most_common()[:10]) #最高频率十个词语


Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\machang\AppData\Local\Temp\jieba.cache


Loading model cost 0.874 seconds.


Prefix dict has been built succesfully.


[('保险', 5005), ('的', 3220), ('人寿保险', 2962), ('什么', 2677), ('吗', 2479), ('是', 2347), ('我', 2056), ('是否', 1862), ('可以', 1704), ('健康', 1513)]


In [10]:
quesition_words=cut(quesitons)
print(quesition_words)

In [11]:
token2 = [''.join(quesition_words[i:i+2]) for i in range(len(quesition_words[:-2]))]
wordcnt2gram=Counter(token2)

In [12]:
def prob2gram(word1,word2):
    if word1+word2 in wordcnt2gram:
        return  wordcnt2gram[word1+word2]/len(wordcnt2gram)
    else:
        return 1/len(wordcnt2gram)


In [13]:
print(prob2gram('我','吗'))
print(prob2gram('有','保险'))
print(prob2gram('保险','吗'))

0.00021101498206372652
0.0002954209748892171
0.021523528170500107


In [14]:
def get_probablity(sentence):
    words = cut(sentence)
    sentence_pro = 1
    for i, word in enumerate(words[:-1]):
        next_ = words[i + 1]
        probability = prob2gram(word, next_)
        sentence_pro *= probability
    return sentence_pro

In [15]:
generator=[]
prob={}
for i in range(0,9):
    str=generate(example_grammar, 'salesman')
    print(str)
    print("prob is ",get_probablity(str))
    prob[str]=get_probablity(str)


你好我是913号,请问你要买车保险吗？
prob is  3.8578588404549156e-45
您好我是5882号,请问你要买财产保险吗？
prob is  3.0647191116159344e-50
女士,你好我是21号,请问你要买车保险吗？
prob is  6.871205013909961e-54
您好我是13号,请问你要买旅行保险吗？
prob is  6.596356813353563e-52
你好我是1号,请问你要买旅行保险吗？
prob is  6.596356813353563e-52
先生,你好我是2856号,您需要买人寿保险吗？
prob is  7.860658535912994e-52
小朋友,您好我是34号,您需要买残疾保险吗？
prob is  2.308806080597102e-53
你好我是41号,您需要买旅行保险吗？
prob is  1.790945230850562e-48
您好我是8号,您需要买残疾保险吗？
prob is  1.296286158089637e-44


In [16]:
c = sorted(prob.items(),key=itemgetter(1))
print(c)

[('女士,你好我是21号,请问你要买车保险吗？', 6.871205013909961e-54), ('小朋友,您好我是34号,您需要买残疾保险吗？', 2.308806080597102e-53), ('您好我是13号,请问你要买旅行保险吗？', 6.596356813353563e-52), ('你好我是1号,请问你要买旅行保险吗？', 6.596356813353563e-52), ('先生,你好我是2856号,您需要买人寿保险吗？', 7.860658535912994e-52), ('您好我是5882号,请问你要买财产保险吗？', 3.0647191116159344e-50), ('你好我是41号,您需要买旅行保险吗？', 1.790945230850562e-48), ('你好我是913号,请问你要买车保险吗？', 3.8578588404549156e-45), ('您好我是8号,您需要买残疾保险吗？', 1.296286158089637e-44)]
